In [1]:
'''
# Automatic Blog Post creator

In this project we will build a blog poster that writes a blog post given something to talk about or discuss.
Additionally, a chariming cover photo will be created!
'''

import random
import string
from pathlib import Path
import os
import requests
import shutil

from git import Repo
from bs4 import BeautifulSoup as Soup
import openai
from PIL import Image

In [2]:
pwd

'/Users/subhasish/Development/GIT/Interstellar/SB-AI-DEV/LLM-GPT/OpenAI/Udemy/Jose Portilla/OpenAI Python API Bootcamp-Learn to use AI, GPT3/SB/04-Auto-Blog-Post'

In [7]:
PATH_TO_BLOG_REPO = Path('/Users/subhasish/Development/GIT/Interstellar/subhasishbiswas.github.io/.git')
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent
PATH_TO_CONTENT = PATH_TO_BLOG/"content"
PATH_TO_CONTENT.mkdir(exist_ok=True, parents=True)

In [4]:
PATH_TO_BLOG_REPO.parent

PosixPath('/Users/subhasish/Development/GIT/Interstellar/subhasishbiswas.github.io')

In [6]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"
PATH_TO_CONTENT

PosixPath('/Users/subhasish/Development/GIT/Interstellar/subhasishbiswas.github.io/content')

In [8]:
def update_blog(commit_message="Update Blogs"):
    #GitPython Repo Location
    repo = Repo(PATH_TO_BLOG_REPO)
    #Add all files to git  -> git add .
    repo.git.add(all=True)
    #Commit all files -> git commit -m "Updated blog"
    repo.index.commit(commit_message)
    #Push to git -> git push origin master
    origin = repo.remote(name='origin')
    origin.push()

In [10]:
def random_string(length):
    letters = string.ascii_lowercase + "\n"
    
    return ''.join(random.choice(letters) for i in range(length))

random_string(10)

'lbmnovuass'

In [13]:
random_blogpost = "Hello Subhasish How are You?"

In [14]:
with open(PATH_TO_BLOG/"index.html", "w") as f:
    f.write(random_blogpost)

In [15]:
update_blog()